In [2]:
import sqlite3
print(sqlite3.version)

2.6.0


C:\Users\Akshat\AppData\Local\Temp\ipykernel_21064\4112040650.py:2: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(sqlite3.version)


In [3]:
def get_connection():
    return sqlite3.connect("library.db")


In [4]:
def create_tables():
    conn = get_connection()
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS books (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            author TEXT NOT NULL,
            isbn TEXT UNIQUE NOT NULL,
             available INTEGER DEFAULT 1
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS borrowers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            phone TEXT
        )
    """)

    conn.commit()
    conn.close()


In [5]:
def add_book(title, author, isbn):
    try:
        conn = get_connection()
        cur = conn.cursor()
        cur.execute("INSERT INTO books (title, author, isbn) VALUES (?, ?, ?)",
                    (title, author, isbn))
        conn.commit()
    except sqlite3.IntegrityError as e:
        print(f"Error: {e}")
    finally:
        conn.close()

def add_borrower(name, email, phone):
    try:
        conn = get_connection()
        cur = conn.cursor()
        cur.execute("INSERT INTO borrowers (name, email, phone) VALUES (?, ?, ?)",
                    (name, email, phone))
        conn.commit()
    except sqlite3.IntegrityError as e:
        print(f"Error: {e}")
    finally:
        conn.close()

In [6]:
def show_available_books():
    conn = get_connection()
    cur = conn.cursor()
    cur.execute("SELECT id, title, author, isbn FROM books WHERE available=1")
    books = cur.fetchall()
    conn.close()

    print("\nAvailable Books:")
    for b in books:
        print(f"{b[0]} | {b[1]} by {b[2]} (ISBN: {b[3]})")

def show_borrowers():
    conn = get_connection()
    cur = conn.cursor()
    cur.execute("SELECT id, name, email, phone FROM borrowers")
    borrowers = cur.fetchall()
    conn.close()

    print("\nBorrowers:")
    for br in borrowers:
        print(f"{br[0]} | {br[1]} | {br[2]} | {br[3]}")

def search_books(keyword):
    conn = get_connection()
    cur = conn.cursor()
    cur.execute("SELECT id, title, author, isbn FROM books WHERE title LIKE ? OR author LIKE ?",
                (f"%{keyword}%", f"%{keyword}%"))
    results = cur.fetchall()
    conn.close()

    print(f"\nSearch Results for '{keyword}':")
    for r in results:
        print(f"{r[0]} | {r[1]} by {r[2]} (ISBN: {r[3]})")


In [7]:
def insert_sample_data():
    conn = get_connection()
    cur = conn.cursor()
    
    # Delete all data from the tables
    cur.execute("DELETE FROM books")
    cur.execute("DELETE FROM borrowers")
    conn.commit()

    sample_books = [
        ("The Alchemist", "Paulo Coelho", "9780061122415"),
        ("To Kill a Mockingbird", "Harper Lee", "9780060935467"),
        ("1984", "George Orwell", "9780451524935"),
        ("Pride and Prejudice", "Jane Austen", "9780141439518"),
        ("The Great Gatsby", "F. Scott Fitzgerald", "9780743273565"),
    ]

    sample_borrowers = [
        ("Alice Johnson", "alice@example.com", "1234567890"),
        ("Bob Smith", "bob@example.com", "9876543210"),
        ("Charlie Brown", "charlie@example.com", "5556667777"),
    ]

    for book in sample_books:
        add_book(*book)

    for borrower in sample_borrowers:
        add_borrower(*borrower)
        
    conn.close()

In [8]:
create_tables()
insert_sample_data()

show_available_books()
show_borrowers()
search_books("George")



Available Books:
1 | The Alchemist by Paulo Coelho (ISBN: 9780061122415)
2 | To Kill a Mockingbird by Harper Lee (ISBN: 9780060935467)
3 | 1984 by George Orwell (ISBN: 9780451524935)
4 | Pride and Prejudice by Jane Austen (ISBN: 9780141439518)
5 | The Great Gatsby by F. Scott Fitzgerald (ISBN: 9780743273565)

Borrowers:
1 | Alice Johnson | alice@example.com | 1234567890
2 | Bob Smith | bob@example.com | 9876543210
3 | Charlie Brown | charlie@example.com | 5556667777

Search Results for 'George':
3 | 1984 by George Orwell (ISBN: 9780451524935)
